In [1]:
import pandas as pd
import seaborn as sns
import json
import re

def generate_dataset():
    with open('stats_without_preprocessing.json','r') as f:
        stats = json.load(f)
    
    df = pd.DataFrame(stats)
    df['preprocessed'] = False
    
    with open('stats_with_preprocessing.json','r') as f:
        stats = json.load(f)

    tmp = pd.DataFrame(stats)
    tmp['preprocessed'] = True
    df = df.append(tmp)
    
    with open('stats.json') as f:
        bert_stats = json.load(f)
    bert_df = pd.DataFrame(bert_stats)
    bert_df['preprocessed'] = False
    
    df = df.append(bert_df)
    df = df.reset_index().drop(columns='index')
    df['pearson'] = df['pearson'].apply(lambda x: round(float(x), 3))
    df['MSE'] = df['MSE'].apply(lambda x: round(float(x), 3))
    
    relevant_cols = set(df.columns) - set(['MSE', 'pearson', 'timestamp'])
    df = df.sort_values(by='timestamp')
    df = df.drop_duplicates(subset=relevant_cols, keep='last')
    df = df.sort_values(by='pearson', ascending=False)
    return df

def get_language_model(x):
    for item in ['ELMo_wiki', 'ELMo_default', 'ELMo_brwac', 'BERT']:
        if item in x:
            return item
    else:
        return None

def get_unk(x):
    if 'unk' in x:
        return True
    else:
        return False 

def get_word_embedding(x):
    if not 'NILC' in x:
        return None
    return ' : '.join(x.rstrip('.model').split('/')[-2:len(x)])

def get_word_embedding_architecture(x):
    if not x:
        return None
    if 'skip' in x:
        return 'skip-gram'
    if 'cbow' in x:
        return 'CBOW'
    return None

def get_word_embedding_size(x):
    if not x:
        return None
    return re.findall(r's\d+.', x)[0].lstrip('s')

def isolate_word_embedding_name(x):
    if not x:
        return None
    return x.split(':')[0]

def retrieve_results(chosen_language='ptbr'):
    df = generate_dataset()
    df['language_model'] = df['test'].apply(get_language_model)
    df['unk'] = df['test'].apply(get_unk)
    df['word_embedding'] = df['test'].apply(get_word_embedding)
    df['word_embedding_architecture'] = df['word_embedding'].apply(get_word_embedding_architecture)
    df['word_embedding_size'] = df['word_embedding'].apply(get_word_embedding_size)
    df['word_embedding'] = df['word_embedding'].apply(isolate_word_embedding_name)
    df = df.rename(columns={'lang': 'dataset_language'})
    df = df[df['dataset_language'] == chosen_language]
    df = df[['dataset_language', 
             'word_embedding', 
             'language_model', 
             'word_embedding_architecture', 
             'word_embedding_size',
            'unk',
            'preprocessed',
            'pearson',
            'MSE']]
    df = df.reset_index().drop(columns='index')
    df = df.sort_values(by='pearson', ascending=False)
    return df

retrieve_results(chosen_language='ptbr').sample(10)

,dataset_language,word_embedding,language_model,word_embedding_architecture,word_embedding_size,unk,preprocessed,pearson,MSE
132,ptbr,wang2vec,None,CBOW,600,True,True,0.443,0.611
90,ptbr,word2vec,None,skip-gram,100,False,True,0.484,0.583
19,ptbr,fasttext,ELMo_wiki,skip-gram,600,False,True,0.592,0.495
106,ptbr,glove,ELMo_wiki,None,50,False,True,0.473,0.591
135,ptbr,glove,None,None,100,True,True,0.440,0.613
14,ptbr,None,BERT,None,None,False,False,0.604,0.483
181,ptbr,fasttext,None,CBOW,300,False,False,0.341,0.672
27,ptbr,word2vec,None,CBOW,1000,False,True,0.580,0.505
117,ptbr,word2vec,None,CBOW,50,True,True,0.458,0.601
188,ptbr,glove,None,None,50,True,False,0.317,0.684


In [2]:
retrieve_results(chosen_language='ptbr').to_csv('propor2020_test_results_ptbr.csv')
retrieve_results(chosen_language='pteu').to_csv('propor2020_test_results_pteu.csv')

In [3]:
def get_preprocessed_diff(df):
    pp_df = df[ df['preprocessed'] == True ]
    pp_df = pp_df[pp_df['language_model'].isnull()]
    non_pp_df = df[df['preprocessed'] == False ]
    non_pp_df = non_pp_df[non_pp_df['language_model'].isnull()]
    dataplot = pp_df.merge(non_pp_df, how='inner', on=['word_embedding', 'language_model', 'word_embedding_architecture', 'word_embedding_size', 'unk'])
    dataplot['pearson_diff'] = dataplot['pearson_x'] - dataplot['pearson_y']
    dataplot = dataplot.sort_values(by='pearson_diff', ascending = False)
    dataplot['pearson_diff'] = dataplot['pearson_diff'].apply(lambda x: round(float(x), 3))
    return dataplot
get_preprocessed_diff(retrieve_results(chosen_language='ptbr')).sample(10)

,dataset_language_x,word_embedding,language_model,word_embedding_architecture,word_embedding_size,unk,preprocessed_x,pearson_x,MSE_x,dataset_language_y,preprocessed_y,pearson_y,MSE_y,pearson_diff
40,ptbr,fasttext,None,skip-gram,50,True,True,0.486,0.580,ptbr,False,0.258,0.710,0.228
63,ptbr,fasttext,None,CBOW,600,False,True,0.367,0.658,ptbr,False,0.359,0.662,0.008
52,ptbr,word2vec,None,skip-gram,50,True,True,0.444,0.610,ptbr,False,0.409,0.633,0.035
30,ptbr,wang2vec,None,skip-gram,100,False,True,0.519,0.556,ptbr,False,0.427,0.622,0.092
12,ptbr,fasttext,None,skip-gram,300,True,True,0.562,0.520,ptbr,False,0.278,0.703,0.284
51,ptbr,wang2vec,None,CBOW,1000,True,True,0.450,0.607,ptbr,False,0.446,0.609,0.004
67,ptbr,fasttext,None,CBOW,100,False,True,0.355,0.665,ptbr,False,0.334,0.676,0.021
50,ptbr,wang2vec,None,CBOW,1000,False,True,0.450,0.607,ptbr,False,0.443,0.611,0.007
54,ptbr,wang2vec,None,CBOW,600,False,True,0.443,0.611,ptbr,False,0.438,0.615,0.005
24,ptbr,word2vec,None,skip-gram,600,False,True,0.542,0.537,ptbr,False,0.475,0.588,0.067


In [4]:
get_preprocessed_diff(retrieve_results(chosen_language='ptbr')).to_csv('diff_preprocessing_ptbr.csv')
get_preprocessed_diff(retrieve_results(chosen_language='pteu')).to_csv('diff_preprocessing_pteu.csv')

In [5]:
def diff_unk(df, status=True):
    tmp_df = df[ df['preprocessed'] == status ]
    tmp_df = tmp_df[tmp_df['language_model'].isnull()]
    dataplot = tmp_df.merge(tmp_df, how='inner', on=['word_embedding', 'language_model', 'word_embedding_architecture', 'word_embedding_size', 'preprocessed'])
    dataplot['same'] = dataplot['pearson_x'].combine(dataplot['pearson_y'], lambda x, y: True if x == y else False)
    dataplot = dataplot[dataplot['same'] == False]
    dataplot = dataplot.drop(columns='same')
    dataplot['pearson_diff'] = dataplot['pearson_x'] - dataplot['pearson_y']
    dataplot = dataplot.sort_values(by='pearson_diff', ascending=False)
    dataplot['pearson_diff'] = dataplot['pearson_diff'].apply(lambda x: round(float(x), 3))
    return dataplot

diff_unk(retrieve_results(chosen_language='pteu'), status=True).sample(10)

,dataset_language_x,word_embedding,language_model,word_embedding_architecture,word_embedding_size,unk_x,preprocessed,pearson_x,MSE_x,dataset_language_y,unk_y,pearson_y,MSE_y,pearson_diff
125,pteu,fasttext,None,CBOW,1000,True,True,0.387,1.003,pteu,False,0.386,1.004,0.001
121,pteu,fasttext,None,CBOW,600,True,True,0.389,1.001,pteu,False,0.388,1.002,0.001
126,pteu,fasttext,None,CBOW,1000,False,True,0.386,1.004,pteu,True,0.387,1.003,-0.001
117,pteu,glove,None,None,50,False,True,0.405,0.980,pteu,True,0.404,0.980,0.001
134,pteu,fasttext,None,CBOW,100,False,True,0.353,1.033,pteu,True,0.354,1.032,-0.001
14,pteu,fasttext,None,skip-gram,1000,True,True,0.563,0.808,pteu,False,0.571,0.796,-0.008
45,pteu,fasttext,None,skip-gram,600,False,True,0.531,0.847,pteu,True,0.525,0.856,0.006
62,pteu,wang2vec,None,skip-gram,300,False,True,0.518,0.858,pteu,True,0.519,0.858,-0.001
34,pteu,fasttext,None,skip-gram,300,True,True,0.532,0.849,pteu,False,0.539,0.839,-0.007
61,pteu,wang2vec,None,skip-gram,300,True,True,0.519,0.858,pteu,False,0.518,0.858,0.001


In [6]:
diff_unk(retrieve_results(chosen_language='pteu'), status=True).to_csv('unk_diff_preprocessed_pteu.csv')
diff_unk(retrieve_results(chosen_language='ptbr'), status=True).to_csv('unk_diff_preprocessed_ptbr.csv')
diff_unk(retrieve_results(chosen_language='pteu'), status=False).to_csv('unk_diff_not_preprocessed_pteu.csv')
diff_unk(retrieve_results(chosen_language='ptbr'), status=False).to_csv('unk_diff_not_preprocessed_ptbr.csv')